##  Movie Recommendation System Data Warehouse Project: Gauranga Kumar Bhuyan

In [37]:
#importing necessary Libraries
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
#importing Data
df = pd.read_csv('movies.csv')

In [39]:
df.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

#### Filling Null Values 

In [40]:
df = df.fillna(' ')

### Done with null values

#### Merging all necessary columns into one.

In [41]:
df['Features'] = df['genres']+' '+df['keywords']+' '+df['tagline']+' '+df['cast']+' '+df['director']

In [42]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,Features
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron,Action Adventure Fantasy Science Fiction cultu...
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski,Adventure Fantasy Action ocean drug abuse exot...
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes,Action Adventure Crime spy based on novel secr...
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan,Action Crime Drama Thriller dc comics crime fi...
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton,Action Adventure Science Fiction based on nove...


### Dropping unnecessary Columns

In [43]:
df.drop(['homepage', 'crew', 'status', 'original_language', 'production_companies', 'production_countries'], axis=1, inplace=True)

In [45]:
df.tail()

,index,budget,genres,id,keywords,original_title,overview,popularity,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count,cast,director,Features
4798,4798,220000,Action Crime Thriller,9367,united states\u2013mexico barrier legs arms pa...,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]","He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez,Action Crime Thriller united states\u2013mexic...
4799,4799,9000,Comedy Romance,72766,,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,2011-12-26,0,85.0,[],A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns,Comedy Romance A newlywed couple's honeymoon...
4800,4800,0,Comedy Drama Romance TV Movie,231617,date love at first sight narration investigati...,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,2013-10-13,0,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith,Comedy Drama Romance TV Movie date love at fir...
4801,4801,0,,126186,,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,2012-05-03,0,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia,A New Yorker in Shanghai Daniel Henney Eli...
4802,4802,0,Documentary,25975,obsession camcorder crush dream girl,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,2005-08-05,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,Brian Herzlinger,Documentary obsession camcorder crush dream gi...


#### Movies already watched by the user

In [46]:
watched_movies = ['Iron Man', 'Bat Man', 'Avengers']

#### Removing those Movies from Our data Frame

In [48]:
df = df[~df['title'].isin(watched_movies)]

In [49]:
#Making those Features into Vector
features = df['Features']
vec = TfidfVectorizer()
feature_vector = vec.fit_transform(features)

In [50]:
#Finding Cosine Similarity
cs = cosine_similarity(feature_vector)


In [51]:
def suggest_movies(movie_name):
    # Find the closest matching movie name in the list
    find_close_match = difflib.get_close_matches(movie_name, df['title'].tolist())
    close_match = find_close_match[0]

    # Find the index of the closest matching movie in the DataFrame
    index_movie = df[df.title == close_match].index[0]

    # Calculate the similarity score between the closest matching movie and all other movies in the DataFrame
    similarity_score = list(enumerate(cs[index_movie]))

    # Sort the movies by similarity score
    sorted_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    # Print the top 30 suggested movies
    suggested_movies = []
    i = 1
    for movie in sorted_movies:
        index = movie[0]
        title_from_index = df.iloc[index]['title']
        if (i < 6):
            suggested_movies.append(title_from_index)
            i += 1
        else:
            break
    
    return suggested_movies

#### Get movie recommendations for the viewer

In [52]:
watched_movies_recommendations = []
for watched_movie in watched_movies:
    watched_movie_recommendations = suggest_movies(watched_movie)
    watched_movies_recommendations.extend(watched_movie_recommendations)

In [53]:
print("Here are some recommended movies for you based on your watch history")
for i, movie in enumerate(set(watched_movies_recommendations)):
    print(f"{i+1}. {movie}")

Here are some recommended movies for you based on your watch history
1. Criminal
2. Captain America: The Winter Soldier
3. Iron Man 2
4. Bad Boys II
5. Avengers: Age of Ultron
6. Riddick
7. There Be Dragons
8. Knight and Day
9. The Avengers
10. Captain America: Civil War
11. Iron Man 3


### We could find 11 movies recommendation for our user on the basis of our pre-existing data.